## EXAMPLE Quadtree

In [ ]:
# import necessary modules
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import path
import time
import xarray as xr
import xugrid as xu

import matplotlib.pyplot as plt

from hydromt_sfincs import SfincsModel
from hydromt_sfincs import utils

from hydromt.log import setuplog

#### Start with a regular grid model

In [ ]:
logger = setuplog("sfincs_regular", log_level=10)
# Initialize SfincsModel Python class with the artifact data catalog which contains publically available data for North Italy
# we overwrite (mode='w+') the existing model in the root directory if it exists
sf = SfincsModel(
    data_libs=["artifact_data"], root="test_regular", mode="w+", logger=logger
)

In [ ]:
# Specify an input dictionary with the grid settings x0,y0,dx,dy,nmax,mmax,rotation and epsg code.
# create SFINCS model with regular grid and characteristics of the input dictionary:
sf.setup_grid(
    x0=318650,
    y0=5040000,
    dx=50.0,
    dy=50.0,
    nmax=107,
    mmax=250,
    rotation=27,
    epsg=32633,
)

In [ ]:
# # show the model grid outline
# # sf.region.boundary.plot(figsize=(6,6))
# _ = sf.plot_basemap(plot_region=True, bmap="sat", zoomlevel=12)

In [ ]:
# # now create a geodataframe that covers a part of the model grid
# datasets_dep = [{"elevtn": "merit_hydro", "zmin": 0.001}, {"elevtn": "gebco"}]

# # Add depth information to modelgrid based on these chosen datasets
# sf.setup_dep(datasets_dep=datasets_dep)
# sf.setup_mask_active(zmin=-5, reset_mask=True)
# sf.setup_mask_bounds(btype="waterlevel", zmax=-5, reset_bounds=True)

In [ ]:
# derive river from hydrography data based on a minimum river length (river_len)
# and minimum upstream area (river_upa)

sf.setup_river_inflow(
    hydrography="merit_hydro", river_len=1000, river_upa=50, keep_rivers_geom=True
)

Create a 200m buffer around the river

In [ ]:
gdf_riv = sf.geoms["rivers_inflow"].copy()
gdf_riv_buf = gdf_riv.assign(geometry=gdf_riv.buffer(200))
gdf_riv_buf["refinement_level"] = 2

Create a 500m buffer around the coastline

In [ ]:
gdf_osm = sf.data_catalog.get_geodataframe("osm_coastlines", bbox=sf.bbox, buffer=0)

# convert polygon to line
gdf_osm_line = gdf_osm.to_crs(sf.crs).boundary

# add a buffer to the line of 500m
gdf_osm_buf = gdf_osm_line.buffer(500)

# clip to model extent (whhy again needed?)
gdf_osm_buf = gdf_osm_buf.intersection(sf.region.geometry)

In [ ]:
# # Make a plot of model
# # note the src points and derived river network
# fig, ax = sf.plot_basemap(variable="dep", plot_bounds=False, bmap="sat")

# # plot gdf_riv on top
# gdf_riv_buf.plot(ax=ax, color="red", linewidth=0.5)
# gdf_osm_buf.plot(ax=ax, color="black", linewidth=0.5)

#### Continue with building a QuadTree model that is refined along the river and coast

In [ ]:
logger = setuplog("sfincs_quadtree", log_level=10)
sf_qt = SfincsModel(
    data_libs=["artifact_data"], root="test_quadtree2", mode="w+", logger=logger
)

In [ ]:
gdf_refinement = gpd.GeoDataFrame(
    {"refinement_level": [1, 2]},
    geometry=[
        gdf_osm_buf.unary_union,
        gdf_riv_buf.unary_union,
    ],
    crs=sf.crs,
)

# traditional way of creating a quadtree grid
sf_qt.setup_grid(
    x0=318650,
    y0=5040000,
    dx=50.0,
    dy=50.0,
    nmax=107,
    mmax=250,
    rotation=27,
    epsg=32633,
    refinement_polygons=gdf_refinement,
)

# alternative way of creating a quadtree grid
# sf_qt.setup_grid_from_region(region={"geom": sf.region}, #area that needs to be covered by the grid
#                             res=50, # set resolution
#                             rotated=True, # when True, rotation is determined to minimize the grid extent
#                             refinement_polygons=gdf_refinement)

# NOTE this grid is smaller since the inactive cells of the regular model are already excluded from the grid

Generate topobathy on the quadtree grid

In [ ]:
datasets_dep = [{"elevtn": "merit_hydro", "zmin": 0.001}, {"elevtn": "gebco"}]

sf_qt.setup_dep(datasets_dep=datasets_dep)

In [ ]:
sf_qt.quadtree.data["dep"].ugrid.plot()

Continue with the mask for the QuadTree grid; we aim to have the same active extent as the regular grid in different ways:
- Based on elevation
- Bu using an include polygon

In [ ]:
# add the gdf mask_include to gdf_refinement as combined gdf
mask_include = gpd.GeoDataFrame(
    pd.concat([sf.region, gdf_refinement], ignore_index=True)
)

# create a new buffer that most includes the open boundary
gdf_osm_buf2 = gdf_osm_buf.buffer(2000, cap_style=3)
open_include = gpd.GeoDataFrame(geometry=gdf_osm_buf2, crs=sf_qt.crs)

In [ ]:
# sf_qt.quadtree.setup_mask(include_polygon=mask_include, open_boundary_polygon=open_include, open_boundary_zmax=-3)

In [ ]:
# sf_qt.setup_mask_active(include_mask=mask_include, all_touched=False)

In [ ]:
# NOTE we lose performance through the SfincsModel, I expect this to come from the data_catalog that parses the geodataframes
# sf_qt.quadtree.setup_mask_active(gdf_include=mask_include, all_touched=False)
sf_qt.quadtree.setup_mask_active(zmin=-5, reset_mask=True, all_touched=False)

In [ ]:
sf_qt.quadtree.data["msk"].ugrid.plot()

In [ ]:
# %matplotlib qt
%matplotlib inline
sf_qt.setup_mask_bounds(
    btype="waterlevel", include_mask=open_include, zmax=-3, reset_bounds=True
)
sf_qt.quadtree.data["msk"].ugrid.plot()

In [ ]:
# %matplotlib qt
%matplotlib inline
sf_qt.setup_mask_bounds(
    btype="waterlevel",
    include_mask=open_include,
    zmax=-3,
    reset_bounds=True,
    connectivity=4,
)
sf_qt.quadtree.data["msk"].ugrid.plot()

In [ ]:
# sf_qt.quadtree.setup_mask_active(model="snapwave", gdf_include= mask_include)#, gdf_exclude= new_exclude)#, gdf_include= include

# sf_qt.quadtree.data["snapwave_msk"].ugrid.plot()
# plt.axis('equal')

In [ ]:
# plot the difference between the mask (original code) and the msk (new code)
# NOTE with all_touched=True, there is a small difference
# (sf_qt.quadtree.data["mask"]-sf_qt.quadtree.data["msk"]).ugrid.plot()

In [ ]:
# drop the mask variable from the quadtree data
# sf_qt.quadtree.data = sf_qt.quadtree.data.drop("mask")

## Now some snapwave functionalities

In [ ]:
sf_qt.setup_mask_active(
    model="snapwave", zmin=-10, zmax=0
)  # , include_mask=gdf_riv_buf)
sf_qt.quadtree.data["snapwave_msk"].ugrid.plot()
plt.axis("equal")

Try situation where we want to directly copy the SFINCS mask to SnapWave:

In [ ]:
# %matplotlib qt
%matplotlib inline
# import importlib
# from hydromt_sfincs import quadtree
# importlib.reload(quadtree)

sf_qt.setup_mask_bounds(
    model="snapwave",
    include_mask=open_include,
    zmax=-4,
    copy_sfincsmask=False,
    connectivity=4,
)
sf_qt.quadtree.data["snapwave_msk"].ugrid.plot()

plt.axis("equal")

In [ ]:
sf_qt.quadtree.data

### Now create a subgrid table for this model

In [ ]:
# sf_qt.setup_subgrid(datasets_dep=datasets_dep, buffer_cells=40)

### Add some random boundary conditions

In [ ]:
# x&y-locations in same coordinate reference system as the grid:
x = [319526, 329195]
y = [5041108, 5046243]

# add to Geopandas dataframe as needed by HydroMT
pnts = gpd.points_from_xy(x, y)
index = [1, 2]  # NOTE that the index should start at one
bnd = gpd.GeoDataFrame(index=index, geometry=pnts, crs=sf.crs)

# In this case we will provide 3 values (periods=3) between the start (tstart=20100201 000000) and the end (tstop=20100201 120000) of the simulation:
time = pd.date_range(
    start=utils.parse_datetime(sf.config["tstart"]),
    end=utils.parse_datetime(sf.config["tstop"]),
    periods=3,
)

# add some water levels
bzs = [[0, 0.25], [0.75, 1.0], [0, 0.25]]

bzspd = pd.DataFrame(index=time, columns=index, data=bzs)

# Actually add it to the SFINCS model class:
sf_qt.setup_waterlevel_forcing(timeseries=bzspd, locations=bnd)

In [ ]:
# # We now use the previously created src discharge points for the regular model
# gdf = sf.forcing["dis"].vector.to_gdf()

# # make up some discharge data
# index = sf.forcing["dis"].index
# dis = np.array([[2.0, 1.0], [5.0, 2.0], [2.0, 1.0]])
# dispd = pd.DataFrame(index=time, columns=index, data=dis)

# # now we call the function setup_discharge_forcing, which adds the discharge forcing to the src points
# sf_qt.setup_discharge_forcing(timeseries=dispd, locations=gdf)

# # # NOTE: the discharge forcing data is now stored in the sf.forcing dictionary
# sf_qt.forcing.keys()

### And save everything we build sofar

In [ ]:
# this is needed for stability?
# TODO check wheterh we have a stable solution in the end; seems to be unstable at the jumps in resolution
sf_qt.config["tspinup"] = 3600
sf_qt.config["alpha"] = 0.4

In [ ]:
sf_qt.config[
    "snapwave_bndfile"
] = "input_locations_ERA5_tmp.bnd"  # org of OuterBanks is in UTM 18N not 17N!
sf_qt.config["snapwave_bhsfile"] = "bhs_ERA5_tmp.bhs"
sf_qt.config["snapwave_btpfile"] = "btp_ERA5_tmp.btp"
sf_qt.config["snapwave_bwdfile"] = "mwd_ERA5_tmp.mwd"
sf_qt.config["snapwave_bdsfile"] = "bds_ERA5_tmp.bds"

In [ ]:
sf_qt.write()

Run and read results, make plots and animation

In [ ]:
import xugrid as xu

uds = xu.open_dataset(r"..\test_quadtree\sfincs_map.nc")

In [ ]:
uds["zb"].ugrid.plot()

In [ ]:
uds["zsmax"].max(dim="timemax").ugrid.plot(vmax=2)

In [ ]:
# plot rough estimation of water depth
h = uds["zsmax"].max(dim="timemax") - uds["zb"]

h = h[h > 0.1]

h.ugrid.plot()

In [ ]:
# create zs plot and save to mod.root/figs/sfincs_zs.mp4
# requires ffmpeg install with "conda install ffmpeg -c conda-forge"
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation

step = 1  # one frame every <step> dtout
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
da_zs = uds["zs"]


def update_plot(i, da_zs, cax_zs):
    da_zsi = da_zs.isel(time=i)
    t = da_zsi.time.dt.strftime("%d-%B-%Y %H:%M:%S").item()
    ax.set_title(f"SFINCS water level {t}")
    cax_zs.set_array(da_zsi.values.ravel())


fig, ax = plt.subplots(figsize=(11, 7))
cax_zs = da_zs.isel(time=0).ugrid.plot(
    ax=ax, vmin=0, vmax=3, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
)
plt.close()  # to prevent double plot

ani = animation.FuncAnimation(
    fig,
    update_plot,
    frames=np.arange(0, da_zs.time.size, step),
    interval=250,  # ms between frames
    fargs=(
        da_zs,
        cax_zs,
    ),
)

# to save to mp4
# ani.save(join(mod.root, 'figs', 'sfincs_h.mp4'), fps=4, dpi=200)

# to show in notebook:
from IPython.display import HTML

HTML(ani.to_html5_video())

In [ ]:
uds["msk"].where(uds["msk"] == 2, np.nan).ugrid.plot()